# NITorch (**N**euro**I**maging Py**Torch**)

# Spatial Tools Demo

Also available in self-contained Colab notebook

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1-dfCosj9XoesFt7byIhp84p2JMUuHxby?usp=sharing "NITorch Spatial Tools Demo")

##### For offline version:

In [ ]:
# GETTING STARTED
# This demo was run on:
# * Ubuntu    18.04.4 LTS / CentOS 7.7.1908
# * CUDA      10.1
# * anaconda  1.7.2
# * gcc       6.3.1
# * pytorch   1.6
#
# To get started, run the following commands in a terminal:
#
#   git clone git@github.com:balbasty/nitorch.git
#   cd nitorch
#   conda env create --file ./conda/nitorch-demo.yml
#   conda activate nitorch-demo
#   pip install .

### Installations

First clone the repo...

In [ ]:
!git clone https://github.com/balbasty/nitorch

Cloning into 'nitorch'...
remote: Enumerating objects: 702, done.
remote: Counting objects: 100% (702/702), done.
remote: Compressing objects: 100% (365/365), done.
remote: Total 3694 (delta 451), reused 522 (delta 315), pack-reused 2992
Receiving objects: 100% (3694/3694), 1.10 MiB | 20.81 MiB/s, done.
Resolving deltas: 100% (2493/2493), done.


Set-up of NITorch and dependencies

In [ ]:
! pip install numpy
! pip install nibabel
! pip install matplotlib
! pip install scipy

In [ ]:
! pip install ./nitorch/

Processing ./nitorch
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for nitorch: filename=nitorch-0.1a0.dev0-cp36-cp36m-linux_x86_64.whl size=4115918 sha256=cb923a216cc90e6aabb9c579bbda8138b5a1e875fd822af20f417684d5e2cb0a
  Stored in directory: /tmp/pip-ephem-wheel-cache-5lpk98h5/wheels/0c/bb/25/d106ef6b7df47bd9f78080b9462f75a2344587550034d540a1
Successfully built nitorch
  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=b4d9320137be17bafb4609d5679d0145e1f3cdc7daaab1ac6b5ea07fed266b02
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


Above cells can be pasted into your own colab notebook for easy install

## GETTING STARTED

First, I will import required packages:

In [ ]:
# Python
import os
import wget
import math
from timeit import default_timer as timer

# Torch / NiBabel
import nibabel as nib
import torch
from torch.nn import functional as F

# NiTorch
from nitorch.core.kernels import smooth
from nitorch.spatial import (affine_matrix_classic, affine_grid, identity_grid,
                             grid_pull, grid_push, voxel_size)
from nitorch.plot import show_slices
from nitorch.core.utils import pad

and get the PyTorch device:

In [ ]:
device_type = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device_type)
if device_type == 'cuda':
    print('GPU: ' + torch.cuda.get_device_name(0) + ', CUDA: ' + str(torch.cuda.is_available()))
else:
    print('CPU')

I will in this demo use an (almost) 1 mm isotropic T1w MRI from the IXI dataset (http://brain-development.org/ixi-dataset/):

In [ ]:
# URL to MRI
url = 'https://www.dropbox.com/s/vsq2yczm1c343wv/IXI002-T1-Guys-0828-MPRAGESEN_-s256_-0301-00003-000001-01.nii?dl=1'

# Path to downloaded image file
pth = 'IXI002-T1-Guys-0828-MPRAGESEN_-s256_-0301-00003-000001-01.nii'

# Download file
if not os.path.exists(pth):
    print('Downloading image...', end='')
    wget.download(url, pth)
    print('done!')

In [ ]:
# Get image data and affine matrix
nii = nib.load(pth)
mat_in = torch.tensor(nii.affine).double().to(device)
dat_in = torch.tensor(nii.get_fdata()).float().to(device)
dat_in_cpu = dat_in.cpu()
dim_in = dat_in.shape
vx_in = voxel_size(mat_in)

# Look at input
fig = show_slices(dat_in)

Finally, I will set some `nitorch` specifics (for detailed documentation look at header of individual nitorch functions):

In [ ]:
bound = 'zero'  # what boundary condition to assume
extrapolate = True  # what to do with voxels that falls outside of the field of view
interpolation = 1  # degree of interpolation

# Make dimensionality compatible with nitorch (input should have dimensions (batch, channels, X, Y, Z))
dat_in = dat_in[None, None, ...]
dat_in_cpu = dat_in_cpu[None, None, ...]

## Example 1: Randomly realign image

This example will apply a random realignment (translation + rotation) to the input image using `nitorch`, comparing doing so on the GPU and the CPU. Note that this example could easily be modified to apply just a flip or a shift.

In [ ]:
# Create an affine matrix that does a random shift in translation and a rotation
p = torch.zeros(6, device=dat_in.device, dtype=torch.float64)
p[:3] = 5*torch.randn(3, device=dat_in.device, dtype=torch.float64)  # translation
p[3:] = 5*torch.randn(3, device=dat_in.device, dtype=torch.float64)*(math.pi/180)  # rotation
mat_out = affine_matrix_classic(p)

# Create an identity grid of size dim_in and apply affine matrix mat_out
grid = affine_grid(mat_out, dim_in).type(dat_in.dtype)[None, ...]
grid_cpu = grid.cpu()

Run on the **GPU**:

In [ ]:
# Pull (resample) input image (dat_in) on the GPU
t = timer()  # Start timer
dat_pull = grid_pull(dat_in, grid,
                     bound=bound, extrapolate=extrapolate, interpolation=interpolation)
print('device={} | elapsed time: {:0.4f} seconds'.format(dat_pull.device, timer() - t))

fig = show_slices(dat_pull[0, 0, ...])


Run on the **CPU**:

In [ ]:
# Pull (resample) input image (dat_in) on the CPU
t = timer()  # Start timer
dat_pull = grid_pull(dat_in_cpu, grid_cpu,
                     bound=bound, extrapolate=extrapolate, interpolation=interpolation)
print('device={} | elapsed time: {:0.4f} seconds'.format(dat_pull.device, timer() - t))

fig = show_slices(dat_pull[0, 0, ...])

## Example 2: Pull isotropic to anisotropic, then push (splat)

This example will make the input (near to 1 mm isotropic) image thick-sliced. It will then use the adjoint of the pull operation to push (splat) the generated thick-sliced image back to its isotropic space.

In [ ]:
# Set output voxel size
vx_out = torch.tensor([1.0, 1.0, 6.0], device=dat_in.device, dtype=torch.float64)

# Get output dimensions and orientation matrix
one = torch.tensor(1.0, device=dat_in.device, dtype=torch.float64)[..., None]
D = torch.diag(torch.cat((vx_out / vx_in, one)))
mat_out = mat_in.mm(D)
mat_out[:3, 3] = mat_out[:3, 3] + 0.5*(vx_out - vx_in)
dim_out = D.inverse()[:3, :3].mm(torch.tensor(dim_in, device=dat_in.device,
                                 dtype=torch.float64).reshape((3, 1))).ceil().squeeze()
dim_out = dim_out.int().cpu().tolist()

# Create an identity grid of size dim_out and apply affine matrix mat_pull
mat_pull = mat_out.solve(mat_in)[0]
grid = affine_grid(mat_pull, dim_out).type(dat_in.dtype)[None, ...]
grid_cpu = grid.cpu()


Run on the **GPU**:

In [ ]:
# Pull (resample) input image (dat_in) on the GPU
t = timer()  # Start timer
dat_pull = grid_pull(dat_in, grid,
                     bound=bound, extrapolate=extrapolate, interpolation=interpolation)
print('device={} | elapsed time: {:0.4f} seconds'.format(dat_pull.device, timer() - t))

fig = show_slices(dat_pull[0, 0, ...])

In [ ]:
# Push the pulled image, on the GPU
t = timer()  # Start timer
dat_push = grid_push(dat_pull, grid, shape=dim_in,
                     bound=bound, extrapolate=extrapolate, interpolation=interpolation)
print('device={} | elapsed time: {:0.4f} seconds'.format(dat_push.device, timer() - t))

fig = show_slices(dat_push[0, 0, ...])


Run on the **CPU**:

In [ ]:
# Pull (resample) input image (dat_in) on the CPU
t = timer()  # Start timer
dat_pull = grid_pull(dat_in_cpu, grid_cpu,
                     bound=bound, extrapolate=extrapolate, interpolation=interpolation)
print('device={} | elapsed time: {:0.4f} seconds'.format(dat_pull.device, timer() - t))

fig = show_slices(dat_pull[0, 0, ...])

In [ ]:
# Push the pulled image, on the GPU
t = timer()  # Start timer
dat_push = grid_push(dat_pull, grid_cpu, shape=dim_in,
                     bound=bound, extrapolate=extrapolate, interpolation=interpolation)
print('device={} | elapsed time: {:0.4f} seconds'.format(dat_push.device, timer() - t))

fig = show_slices(dat_push[0, 0, ...])


## Example 3: Apply random deformation

This example will apply a random (nonlinear) deformation to the input image. The nonlinear deformation is generated by simply sampling from a Gaussian and then smoothing this sampled data.

In [ ]:
# Parameters of random displacement
kernel = 'gauss'
sd = 10
fwhm = 5

# Create an identity grid of size dim_in
grid = identity_grid(dim_in, device=dat_in.device, dtype=dat_in.dtype)

# Simulate a random displacement
df = sd*torch.randn(grid.shape, device=dat_in.device, dtype=dat_in.dtype)

# Make smoothing kernel
smo = smooth((kernel,) * 3, fwhm=(fwhm,) * 3, device=dat_in.device, dtype=dat_in.dtype, sep=False)

# Padding amount for subsequent convolution
p = (torch.tensor(smo.shape[2:]) - 1)//2
p = tuple(p.int().cpu().tolist())

# Smooth displacement with Gaussian kernel (by convolving)
for d in range(3):
    tmp = pad(df[..., d], p, side='both')
    tmp = F.conv3d(tmp[None, None, ...], smo)[0, 0, ...]
    df[..., d] = tmp

# Apply the displacement to the identity grid to make the random deformation
grid = grid + df

grid = grid[None, ...]
grid_cpu = grid.cpu()

Run on the **GPU**:

In [ ]:
# Pull (resample) input image (dat_in) on the GPU
t = timer()  # Start timer
dat_pull = grid_pull(dat_in, grid,
                     bound=bound, extrapolate=extrapolate, interpolation=interpolation)
print('device={} | elapsed time: {:0.4f} seconds'.format(dat_pull.device, timer() - t))

fig = show_slices(dat_pull[0, 0, ...])

Run on the **CPU**:

In [ ]:
# Pull (resample) input image (dat_in) on the CPU
t = timer()  # Start timer
dat_pull = grid_pull(dat_in_cpu, grid_cpu,
                     bound=bound, extrapolate=extrapolate, interpolation=interpolation)
print('device={} | elapsed time: {:0.4f} seconds'.format(dat_pull.device, timer() - t))

fig = show_slices(dat_pull[0, 0, ...])
